In [2]:
!pip install accelerate>=0.12.0
!pip install -U "ray[serve]"  # installs Ray + dependencies for Ray Serve
!pip install -U slack-bolt
!pip install faiss-cpu
!pip install atlassian-python-api
!pip install numpy


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Using cached ray-2.3.1-cp39-cp39-manylinux2014_x86_64.whl (58.6 MB)
     |████████████████████████████████| 58.6 MB 4.8 MB/s eta 0:00:01
  Using cached slack_bolt-1.17.2-py2.py3-none-any.whl (194 kB)
     |████████████████████████████████| 194 kB 5.2 MB/s eta 0:00:01


In [4]:
!pip install transformers[torch]==4.25.1 

In [38]:
!pip install torch

!pip install transformers[torch]==4.25.1

In [40]:
import sys
print(sys.path)
print(sys.executable)

['/home/fan', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages']
/usr/bin/python3


In [42]:
!echo $SHELL
!echo "export PATH=\$PATH:/home/fan/.local/bin" >> "$HOME/.bashrc"
!source "$HOME/.bashrc"
print(sys.path)

!pip freeze | grep atlassian

/bin/bash
['/home/fan', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages']
atlassian-python-api==3.36.0


In [3]:
from atlassian import Confluence
import os

# Set up Confluence API connection
confluence = Confluence(
url='https://advendio.atlassian.net',
)

space_key = "SO"
pages = confluence.get_all_pages_from_space(space_key)

# Create a directory to store the downloaded pages
if not os.path.exists('advendio_pages'):
    os.makedirs('advendio_pages')
# Download each page
for page in pages:
    page_id = page['id']
    page_title = page['title']
    page_filename = page_title.replace(' ', '_') + '.html'
    page_content = confluence.get_page_by_id(page_id, expand='body.storage')['body']['storage']['value']
    try:
        with open('advendio_pages/' + page_filename, 'w') as f:
            f.write(page_content)
    except:
        pass
    print('Downloaded:', page_filename)


Downloaded: 7.3.1_Create_a_Connection_with_a_Third_Party_System_(e.g._AdServer,_DSP,_SSP).html
Downloaded: 7.3_Connecting_Third_Party_Systems.html
Downloaded: 7.4.3_Edit_Salesforce_Layouts.html
Downloaded: 7.2_Setup_ADvendio_Features.html
Downloaded: 7.1.4_Edit_Your_Company_Information.html
Downloaded: 7.1.3_Manage_Users_and_Licenses.html
Downloaded: 7.1.8_Currency_Setting.html
Downloaded: 7.5.1_Configure_your_Proposal_Layout.html
Downloaded: 7.1.5_Edit_the_Homepage_Layout.html
Downloaded: 7.5.8_Edit_Visit_Report_Layout.html
Downloaded: 7.5_PDF_Layout_Configuration.html
Downloaded: 7.4.2_Manage_Approvals.html
Downloaded: 7.4.1_Create_Workflows.html
Downloaded: 7.4_Standard_Salesforce_Admin_Basics.html
Downloaded: 3.2.3_Predefined_AdServer_Targeting_for_Products:_Connect_Ad_Specs_via_Ad_Server_IDs.html
Downloaded: 2.1.2_Create_an_Account_with_Contacts.html
Downloaded: 4.6.1_Create_a_PDF_Document.html
Downloaded: 4.2.1_Media_Campaign_Details.html
Downloaded: 4.6.4_Document_Type_in_Media_

In [10]:
!pip install lxml

     |████████████████████████████████| 7.1 MB 4.7 MB/s eta 0:00:01

(ServeController pid=12909) INFO 2023-04-22 04:53:52,382 controller 12909 deployment_state.py:1333 - Adding 1 replica to deployment 'DocumentVectorDB'.


     |████████████████████████████████| 7.1 MB 4.7 MB/s 


(ServeReplica:DocumentVectorDB pid=14078) 
(ServeReplica:DocumentVectorDB pid=14078) The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
(ServeReplica:DocumentVectorDB pid=14078) The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
(ServeReplica:DocumentVectorDB pid=14078) The class this function is called from is 'DPRContextEncoderTokenizer'.
(ServeController pid=12909) WARNING 2023-04-22 04:54:22,421 controller 12909 deployment_state.py:1633 - Deployment DocumentVectorDB has 1 replicas that have taken more than 30s to initialize. This may be caused by a slow __init__ or reconfigure method.


In [11]:
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import VisionEncoderDecoderModel, ViTImageProcessor
from transformers import AutoModelForSeq2SeqLM

from fastapi import FastAPI, Request
from ray import serve
from slack_bolt.async_app import AsyncApp
from slack_bolt.adapter.fastapi.async_handler import AsyncSlackRequestHandler
from slack_bolt.adapter.starlette.handler import SlackRequestHandler
import requests
from slack_sdk.signature import SignatureVerifier
import ray
import asyncio

import logging
import numpy as np

from typing import List
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from bs4 import BeautifulSoup
import os
import faiss

@serve.deployment(ray_actor_options={"num_gpus": 0})
class DocumentVectorDB:
    def __init__(self, 
                 question_encoder_model: str = "facebook/dpr-question_encoder-single-nq-base",
                 context_encoder: str = "facebook/dpr-ctx_encoder-single-nq-base"
                 ):
        self.token_limit = 512

        self.documents = self.format_documents()
        self.question_encoder = DPRQuestionEncoder.from_pretrained(
            question_encoder_model)
        self.question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(
            question_encoder_model)
        self.context_encoder = DPRContextEncoder.from_pretrained(
            context_encoder)
        self.context_tokenizer = DPRContextEncoderTokenizer.from_pretrained(
            context_encoder)
        count = self.index_documents(self.documents)
        print("document count:{}".format(count))
      
    def index_documents(self, documents: List[str]) -> int:
        # Encode the documents
        encoded_documents = self.context_tokenizer(
            self.documents, 
            return_tensors="pt", 
            padding=True, 
            truncation=True, 
            max_length=self.token_limit)
        document_embeddings = self.context_encoder(**encoded_documents).pooler_output

        document_embeddings = document_embeddings.detach().numpy()
        document_embeddings=np.ascontiguousarray(document_embeddings)
        # Create Faiss Index
        vector_dimension = document_embeddings.shape[1]
        print("vector dimension:{}".format(vector_dimension))
        self.index = faiss.IndexFlatL2(vector_dimension)
        faiss.normalize_L2(document_embeddings)
        self.index.add(document_embeddings)
        return self.index.ntotal

    def insert_documents(self) -> List[str]:
        """
        store some whre
        """
        pass

    def encode_questions(self, query: str) -> List[torch.Tensor]:
        encoded_query = self.question_tokenizer(query, return_tensors="pt")
        query_embedding = self.question_encoder(
            **encoded_query).pooler_output.detach().numpy()
        query_embedding = np.ascontiguousarray(query_embedding)
        return query_embedding

    def query_documents(self, query: str) -> str:
        # Encode the query
        query_embedding = self.encode_questions(query)
        _, idx = self.index.search(query_embedding, 4)
        doc = self.documents[idx[0][0]]
        print("query result: index={}, doc={}".format(idx[0][0], doc))
        return doc

    def format_documents(self):
        documents = []
        for filename in os.listdir('advendio_pages'):
            f = os.path.join('advendio_pages', filename)
            with open(f, 'r', encoding='utf-8') as file:
                html_content = file.read()
                soup = BeautifulSoup(html_content, "lxml")

                text_content = soup.get_text(separator=" ", strip=True)
                documents.append(text_content)
        return documents

In [47]:
@serve.deployment(ray_actor_options={"num_gpus": 0})
class RAGConversationBot:
    def __init__(self, 
                 db: DocumentVectorDB, 
                 model: str = "databricks/dolly-v2-3b"):
        self.model = model
        self.db = db

    async def prompt(self, input: str) ->str:
        context_ref = await self.db.query_documents.remote(input)
        context = await context_ref
        assert isinstance(context, str)
        return "{} \n context: {}\n output:".format(input, context)
         
    # Change this method to be an async function
    async def generate_text(self, input_text: str) -> str:
        generator = pipeline(model=self.model, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")
        prompt_text = await self.prompt(input_text)
        assert isinstance(prompt_text, str)
        return generator(prompt_text)[0]

    async def __call__(self, http_request: Request) -> str:
        input_text: str = await http_request.json()
        return await self.generate_text(input_text)

# 2: Deploy the model.
# serve.run(RAGConversationBot.bind(DocumentVectorDB.bind()))

In [19]:
!pip install --upgrade gradio

  Using cached gradio-3.27.0-py3-none-any.whl (17.3 MB)
     |████████████████████████████████| 17.3 MB 4.9 MB/s eta 0:00:01


In [25]:
# 2: Deploy the model.
# serve.run(RAGConversationBot.bind(DocumentVectorDB.bind()))

(ServeController pid=12909) INFO 2023-04-22 05:26:02,137 controller 12909 deployment_state.py:1222 - Stopping 1 replicas of deployment 'DocumentVectorDB' with outdated versions.
(ServeController pid=12909) INFO 2023-04-22 05:26:02,140 controller 12909 deployment_state.py:1222 - Stopping 1 replicas of deployment 'RAGConversationBot' with outdated versions.
(ServeController pid=12909) INFO 2023-04-22 05:26:04,325 controller 12909 deployment_state.py:1333 - Adding 1 replica to deployment 'RAGConversationBot'.
(ServeController pid=12909) INFO 2023-04-22 05:26:22,395 controller 12909 deployment_state.py:1333 - Adding 1 replica to deployment 'DocumentVectorDB'.
(ServeReplica:DocumentVectorDB pid=16026) The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
(ServeReplica:DocumentVectorDB pid=16026) The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
(ServeRepl

(ServeReplica:DocumentVectorDB pid=16026) vector dimension:768
(ServeReplica:DocumentVectorDB pid=16026) document count:49


RayServeSyncHandle(deployment='RAGConversationBot')

In [54]:
example_text = "How is an Ads event stored in ADvendio?"

import requests
import gradio as gr
from ray.serve.gradio_integrations import GradioServer
from ray.serve.gradio_integrations import GradioIngress

@serve.deployment(route_prefix="/")
class MyGradioServer(GradioIngress):
    def __init__(self, convBot: RAGConversationBot):
        self._d1 = convBot

        input_text = gr.inputs.Textbox(
            label="Questions:", default=example_text)
        input_urls = gr.inputs.Textbox(label="Knowledge base URLs:")
        output_text = gr.outputs.Textbox(label="Answer:")
        output_rating = gr.inputs.Radio(
            ["Thumbs Up", "Thumbs Down"], label="Rate the answer")

        super().__init__(lambda: gr.Interface(
        fn=self.fanout,
        inputs=[input_text, input_urls],
        outputs=[output_text, output_rating],
        title="Augmented Retrieval"))


    async def fanout(self, text, urls):
        refs = await asyncio.gather(self._d1.generate_text.remote(text))
        res = ray.get(refs)
        return res

app = MyGradioServer.bind(RAGConversationBot.bind(DocumentVectorDB.bind()))

serve.run(app)

# def gradio_builder():
    
#     model = RAGConversationBot()

#     def translate_text(input_text, input_urls):
#         # 3: Query the deployment and print the result.
#         response = model.generate_text(input_text)
#         return response

#     input_text = gr.inputs.Textbox(label="Questions:", default=example_text)
# #     input_urls = gr.inputs.Textbox(label="Knowledge base URLs:")
#     output_text = gr.outputs.Textbox(label="Answer:")
#     output_rating = gr.inputs.Radio(["Thumbs Up", "Thumbs Down"], label="Rate the answer")

#     return gr.Interface(
#         fn=translate_text,
#         inputs=[input_text], #, input_urls],
#         outputs=[output_text, output_rating],
#         title="Augmented Retrieval"
#     )


# iface = gradio_builder()
# iface.launch(share=True)

(ServeController pid=12909) INFO 2023-04-22 06:23:19,322 controller 12909 deployment_state.py:1222 - Stopping 1 replicas of deployment 'DocumentVectorDB' with outdated versions.
(ServeController pid=12909) INFO 2023-04-22 06:23:19,324 controller 12909 deployment_state.py:1222 - Stopping 1 replicas of deployment 'RAGConversationBot' with outdated versions.
(ServeController pid=12909) INFO 2023-04-22 06:23:19,327 controller 12909 deployment_state.py:1222 - Stopping 1 replicas of deployment 'MyGradioServer' with outdated versions.
(ServeController pid=12909) INFO 2023-04-22 06:23:21,519 controller 12909 deployment_state.py:1333 - Adding 1 replica to deployment 'DocumentVectorDB'.
(ServeController pid=12909) INFO 2023-04-22 06:23:21,536 controller 12909 deployment_state.py:1333 - Adding 1 replica to deployment 'RAGConversationBot'.
(ServeController pid=12909) INFO 2023-04-22 06:23:21,553 controller 12909 deployment_state.py:1333 - Adding 1 replica to deployment 'MyGradioServer'.
(ServeRepl

(ServeReplica:DocumentVectorDB pid=19357) vector dimension:768
(ServeReplica:DocumentVectorDB pid=19357) document count:49


RayServeSyncHandle(deployment='MyGradioServer')

In [55]:
response = requests.get("http://127.0.0.1:8000/", json=english_text)
response.text

'<!DOCTYPE html>\n<html\n\tlang="en"\n\tstyle="\n\t\tmargin: 0;\n\t\tpadding: 0;\n\t\tmin-height: 100%;\n\t\tdisplay: flex;\n\t\tflex-direction: column;\n\t"\n>\n\t<head>\n\t\t<meta charset="utf-8" />\n\t\t<meta\n\t\t\tname="viewport"\n\t\t\tcontent="width=device-width, initial-scale=1, shrink-to-fit=no, maximum-scale=1"\n\t\t/>\n\n\t\t\n\t\t<meta property="og:url" content="https://gradio.app/" />\n\t\t<meta property="og:type" content="website" />\n\t\t<meta property="og:image" content="" />\n\t\t<meta property="og:title" content="Augmented Retrieval" />\n\t\t<meta\n\t\t\tproperty="og:description"\n\t\t\tcontent=""\n\t\t/>\n\t\t<meta name="twitter:card" content="summary_large_image" />\n\t\t<meta name="twitter:creator" content="@teamGradio" />\n\t\t<meta name="twitter:title" content="Augmented Retrieval" />\n\t\t<meta\n\t\t\tname="twitter:description"\n\t\t\tcontent=""\n\t\t/>\n\t\t<meta name="twitter:image" content="" />\n\n\t\t<script>\n\t\t\twindow.dataLayer = window.dataLayer || []

(HTTPProxyActor pid=12977) INFO 2023-04-22 06:26:21,062 http_proxy 10.128.0.7 http_proxy.py:373 - GET / 200 27.0ms
(ServeReplica:MyGradioServer pid=19360) INFO 2023-04-22 06:26:21,057 MyGradioServer MyGradioServer#YTsmOO replica.py:518 - HANDLE __call__ OK 15.9ms
